In [3]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import plotly.express as px

# 设置随机种子以确保可重复性
random.seed(42)
np.random.seed(42)

# 公司名称列表
company_names = [
    "湖北国贸能源化工有限公司",
    "湖北国贸金属矿产有限公司",
    "湖北国贸汽车有限公司",
    "湖北国际贸易集团有限公司",
    "湖北国贸农产品有限公司"
]

# 客户名称列表
customer_names = [f"客户{i}" for i in range(1, 51)]

# 账龄区间列表
aging_intervals = ["1-30天", "31-60天", "61-90天", "91-120天", "121-180天", "180天以上"]

# 生成数据
n_rows = 1000
now = datetime.now()

data = {
    "公司名称": np.random.choice(company_names, n_rows),
    "客户名称": np.random.choice(customer_names, n_rows),
    "账款金额": np.random.uniform(10000, 1000000, n_rows).round(2),
    "账单日期": [(now - timedelta(days=random.randint(1, 365))).strftime("%Y-%m-%d") for _ in range(n_rows)]
}

# 创建 DataFrame
df = pd.DataFrame(data)

# 计算账龄
df['账单日期'] = pd.to_datetime(df['账单日期'])
df['账龄天数'] = (now - df['账单日期']).dt.days

# 添加账龄区间
def get_aging_interval(days):
    if days <= 30:
        return "1-30天"
    elif days <= 60:
        return "31-60天"
    elif days <= 90:
        return "61-90天"
    elif days <= 120:
        return "91-120天"
    elif days <= 180:
        return "121-180天"
    else:
        return "180天以上"

df['账龄区间'] = df['账龄天数'].apply(get_aging_interval)

# 创建公司名称简称的字典
company_short_names = {
    "湖北国贸能源化工有限公司": "国贸能化",
    "湖北国贸金属矿产有限公司": "国贸金属矿",
    "湖北国贸汽车有限公司": "国贸汽车",
    "湖北国际贸易集团有限公司": "国贸集团",
    "湖北国贸农产品有限公司": "国贸农产品"
}

# 将简称添加到DataFrame中
df['公司简称'] = df['公司名称'].map(company_short_names)

def set_mobile_layout(fig, title, x_title, y_title):
    fig.update_layout(
        autosize=False,
        width=360,
        height=300,
        margin=dict(l=10, r=10, t=60, b=40),
        title_font_size=16,
        font=dict(size=12),
        title=dict(text=title, x=0.3),
        xaxis=dict(
            title=x_title,
            tickangle=-45,
            title_font=dict(size=14),
            tickfont=dict(size=10)
        ),
        yaxis=dict(
            title=y_title,
            title_font=dict(size=14),
            tickfont=dict(size=10),
            tickformat='.2s'
        )
    )
    return fig

def calculate_bar_width(n_bars, chart_width=360, min_width=0.5, max_width=0.8):
    ideal_width = (chart_width * 0.8) / (n_bars + 1)
    return max(min(ideal_width / chart_width, max_width), min_width)

# 例子1: 集团用户查询各公司的账龄情况
df_company_aging = df.groupby(['公司名称', '账龄区间'])['账款金额'].sum().unstack(fill_value=0).reset_index()
df_company_aging['公司简称'] = df_company_aging['公司名称'].map(company_short_names)

fig1 = px.bar(df_company_aging, x='公司简称', y=aging_intervals, barmode='stack')
fig1 = set_mobile_layout(fig1, 
                         '各公司账龄分布', 
                         '公司名称', '账款金额 (元)')
fig1.update_xaxes(type='category', categoryorder='total descending')
fig1.update_traces(marker_line_width=1.5, opacity=0.6)
fig1.show()

# 例子2: 公司用户查询客户账龄情况
company = "湖北国贸能源化工有限公司"
df_company = df[df['公司名称'] == company]
df_customer_aging = df_company.groupby('客户名称')['账款金额'].sum().sort_values(ascending=False).head(10).reset_index()

fig2 = px.bar(df_customer_aging, x='客户名称', y='账款金额')
fig2 = set_mobile_layout(fig2, 
                         f'{company_short_names[company]}<br>客户账款金额Top10', 
                         '客户名称', '账款金额 (元)')
fig2.update_xaxes(type='category', categoryorder='total descending')
fig2.update_traces(marker_color='rgb(255,165,0)', marker_line_color='rgb(255,140,0)',
                   marker_line_width=1.5, opacity=0.6, 
                   width=calculate_bar_width(len(df_customer_aging)))
fig2.show()

# 例子3: 集团用户按条件查询特定公司的帐龄情况
company = "湖北国贸金属矿产有限公司"
df_company = df[df['公司名称'] == company]
df_customer_aging_filtered = df_company[df_company['账龄天数'] > 90].groupby(['客户名称', '账龄区间'])['账款金额'].sum().unstack(fill_value=0).reset_index()
df_customer_aging_filtered['总账款'] = df_customer_aging_filtered[['91-120天', '121-180天', '180天以上']].sum(axis=1)
df_customer_aging_filtered = df_customer_aging_filtered.sort_values('总账款', ascending=False).head(10)

fig3 = px.bar(df_customer_aging_filtered, x='客户名称', 
              y=['91-120天', '121-180天', '180天以上'],
              title=f'{company_short_names[company]}<br>账龄超过90天的前10名客户',
              labels={'value': '账款金额 (元)', 'variable': '账龄区间'},
              color_discrete_sequence=['#FFA07A', '#FF7F50', '#FF4500'])
fig3 = set_mobile_layout(fig3, 
                         f'{company_short_names[company]}<br>账龄超过90天的前10名客户', 
                         '客户名称', '账款金额 (元)')
fig3.update_xaxes(type='category', categoryorder='total descending')
fig3.update_layout(barmode='stack', showlegend=True, 
                   legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))
fig3.show()

# 例子4: 集团用户查询特定账龄区间的情况
df_aging_91_plus = df[df['账龄区间'].isin(['91-120天', '121-180天', '180天以上'])]
df_company_aging_91_plus = df_aging_91_plus.groupby(['公司名称', '账龄区间'])['账款金额'].sum().unstack(fill_value=0).reset_index()
df_company_aging_91_plus['总账款'] = df_company_aging_91_plus[['91-120天', '121-180天', '180天以上']].sum(axis=1)
df_company_aging_91_plus = df_company_aging_91_plus.sort_values('总账款', ascending=False)
df_company_aging_91_plus['公司简称'] = df_company_aging_91_plus['公司名称'].map(company_short_names)

fig4 = px.bar(df_company_aging_91_plus, x='公司简称', 
              y=['91-120天', '121-180天', '180天以上'],
              title='各公司账龄91天以上的账款金额',
              labels={'value': '账款金额 (元)', 'variable': '账龄区间'},
              color_discrete_sequence=['#FFA07A', '#FF7F50', '#FF4500'])
fig4 = set_mobile_layout(fig4, 
                         '各公司账龄91天以上<br>的账款金额', 
                         '公司名称', '账款金额 (元)')
fig4.update_xaxes(type='category', categoryorder='total descending')
fig4.update_layout(barmode='stack', showlegend=True, 
                   legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))
fig4.show()

# 例子5: 公司用户查询长账龄客户
company = "湖北国贸能源化工有限公司"
df_company = df[df['公司名称'] == company]
df_long_aging_customers = df_company[df_company['账龄区间'] == '180天以上'].groupby('客户名称')['账款金额'].sum().sort_values(ascending=False).reset_index()

fig5 = px.bar(df_long_aging_customers, x='客户名称', y='账款金额')
fig5 = set_mobile_layout(fig5, 
                         f'{company_short_names[company]}<br>账龄超过180天的客户', 
                         '客户名称', '账款金额 (元)')
fig5.update_xaxes(type='category', categoryorder='total descending')
fig5.update_traces(marker_color='rgb(255,165,0)', marker_line_color='rgb(255,140,0)',
                   marker_line_width=1.5, opacity=0.6, 
                   width=calculate_bar_width(len(df_long_aging_customers)))
fig5.show()

# 例子6: 集团用户比较不同账龄区间的账款金额
df_aging_distribution = df.groupby('账龄区间')['账款金额'].sum().reset_index()

fig6 = px.bar(df_aging_distribution, x='账龄区间', y='账款金额')
fig6 = set_mobile_layout(fig6, 
                         '不同账龄区间的<br>账款金额分布', 
                         '账龄区间', '账款金额 (元)')
fig6.update_xaxes(type='category', categoryorder='array', categoryarray=aging_intervals)
fig6.update_traces(marker_color='rgb(255,165,0)', marker_line_color='rgb(255,140,0)',
                   marker_line_width=1.5, opacity=0.6, 
                   width=calculate_bar_width(len(df_aging_distribution)))
fig6.show()

# 例子7: 公司用户查询特定客户的账龄分布
company = "湖北国贸能源化工有限公司"
df_company = df[df['公司名称'] == company]
customer = df_company['客户名称'].iloc[0]  # 选择该公司的第一个客户
df_company_customer = df_company[df_company['客户名称'] == customer]
df_customer_aging_distribution = df_company_customer.groupby('账龄区间')['账款金额'].sum().reset_index()

fig7 = px.pie(df_customer_aging_distribution, values='账款金额', names='账龄区间', title=f'{company_short_names[company]}<br>{customer}的账龄分布')
fig7.update_traces(textposition='inside', textinfo='percent+label')
fig7.update_layout(
    autosize=False,
    width=360,
    height=300,
    margin=dict(l=10, r=10, t=60, b=40),
    title_font_size=16,
    font=dict(size=12),
    title=dict(x=0.3)
)
fig7.show()

KeyError: "['1-30天', '31-60天', '61-90天'] not in index"